<a href="https://colab.research.google.com/github/mohitvaishnav143/105solpython/blob/main/PRO_C115_Project_Boilerplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the dataset from Github repository
!git clone https://github.com/procodingclass/product_dataset.git

Cloning into 'product_dataset'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), 3.08 MiB | 7.23 MiB/s, done.


In [2]:
# Make a dataframe using the 'pandas' module
import pandas as pd
dataframe = pd.read_excel('/content/product_dataset/updated_product_dataset.xlsx')
print(dataframe.head())

    Emotion                                               Text
0  Positive  close approximation red octane mat bought one ...
1   Neutral  little lumpy mat great foam padding itâ€™s use...
2  Positive  great pad love ddr not want metal pad get work...
3  Positive  excellent pad great product highly responsive ...
4  Positive  awesome great ddr pad works perfectly pc stepm...


In [3]:
# Get unique emotions from the 'Emotion' column in the Dataset
dataframe["Emotion"].unique()

array(['Positive', 'Neutral', 'Negative'], dtype=object)

In [4]:
# Add Labels to the dataset
encode_emotions = {"Neutral": 0, "Positive": 1, "Negative": 2}

In [5]:
# replace the emotions with the Label
dataframe.replace(encode_emotions, inplace = True)
dataframe.head()

,Emotion,Text
0,1,close approximation red octane mat bought one ...
1,0,little lumpy mat great foam padding itâ€™s use...
2,1,great pad love ddr not want metal pad get work...
3,1,excellent pad great product highly responsive ...
4,1,awesome great ddr pad works perfectly pc stepm...


In [6]:
# Convert the dataframe into list for easy processing
training_sentences = []
training_labels = []

for i in range(len(dataframe)):
  sentence = dataframe.loc[i, "Text"]
  training_sentences.append(sentence)
  label = dataframe.loc[i, "Emotion"]
  training_labels.append(label)

# printing 10th element
training_sentences[10], training_labels[10]


('arrived early included blank case wont able test game get switch tried brothers device recognized card smash digital download already arrived ahead schedule well packed even blank case wasnt expectingread full review',
 1)

In [7]:
# Tokenize and pad your data
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [8]:
# Converting to numpy array
import numpy as np

training_padded = np.array(training_padded)
training_labels = np.array(training_labels)

In [14]:
# import the necessary modules from tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.layers import Conv1D,Dropout,MaxPooling1D


# Create the model

model = tf.keras.Sequential([
    Embedding(vocab_size,embedding_dim,input_length=max_length),
    Dropout(0.2),

   Conv1D(filters=256,kernel_size=3,activation="relu"),
   MaxPooling1D(pool_size =3),
    
   Conv1D(filters=128,kernel_size=3,activation="relu"),
   MaxPooling1D(pool_size =3),

   LSTM(128),

   Dense(128,activation="relu"),
   Dropout(0.2),
   Dense(64,activation="relu"),
   Dense(6,activation="softmax")
])


# Compile the model
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [15]:
# Get model Summary before training
num_epochs=5
history=model.fit(training_padded,training_labels,epochs=num_epochs,verbose=2)

Epoch 1/5
671/671 - 43s - loss: 0.3363 - accuracy: 0.9232 - 43s/epoch - 64ms/step
Epoch 2/5
671/671 - 41s - loss: 0.1848 - accuracy: 0.9420 - 41s/epoch - 61ms/step
Epoch 3/5
671/671 - 44s - loss: 0.1410 - accuracy: 0.9530 - 44s/epoch - 65ms/step
Epoch 4/5
671/671 - 38s - loss: 0.1152 - accuracy: 0.9610 - 38s/epoch - 56ms/step
Epoch 5/5
671/671 - 36s - loss: 0.0986 - accuracy: 0.9655 - 36s/epoch - 54ms/step


In [16]:
# Train your Model
model.save("Text_Emotion.h5")


In [17]:
# Save your Model
from math import trunc
sentence = ["the weather was so hot i am not feeling well", "I had a bad day at school. i got hurt while playing football"]
sequences=tokenizer.texts_to_sequences(sentence)

padded=pad_sequences(sequences,maxlen=max_length,
                     padding=padding_type,truncating=trunc_type)

result=model.predict(padded)

predict_class=np.argmax(result,axis=1)

predict_class

1/1 [==============================] - 1s 664ms/step


array([0, 0])